In [20]:
import java.util.stream.*
import java.util.concurrent.*

In [29]:
//cpu核心数
println(Runtime.getRuntime().availableProcessors())
//最大线程数
println(ForkJoinPool.getCommonPoolParallelism())
//当前线程数
println(ForkJoinPool.commonPool().poolSize)

8
7
5


In [30]:
val dished = IntStream.range(0,7)
    .mapToObj({
        CompletableFuture.runAsync(
                {
                    println(it.toString()+":"+Thread.currentThread().name+":"+System.currentTimeMillis())
                    Thread.sleep(2000)
                }
            )
    })
    .toList()

val start = System.currentTimeMillis()
CompletableFuture.allOf(*dished.toTypedArray()).join()
println(System.currentTimeMillis() - start)

1:ForkJoinPool.commonPool-worker-14:1662173217631
4:ForkJoinPool.commonPool-worker-10:1662173217632
3:ForkJoinPool.commonPool-worker-9:1662173217631
2:ForkJoinPool.commonPool-worker-13:1662173217631
0:ForkJoinPool.commonPool-worker-12:1662173217631
6:ForkJoinPool.commonPool-worker-20:1662173217632
5:ForkJoinPool.commonPool-worker-19:1662173217632
2012
